Information Retrieval (Bilgi Getirimi) Nedir?
Bilgi getirimi, kullanıcının sorduğu bir sorguya karşılık olarak büyük bir belge koleksiyonu içerisinden en alakalı belgeleri bulma sürecidir. Bu işlem özellikle:

Arama motorlarında (Google, Bing),

Soru-cevap sistemlerinde,

Belgelerden bilgi çekmede,

Chatbot'larda
kullanılır.

Temel Adımlar:
Belgeler ve sorgu sayısallaştırılır.

Her biri vektör haline getirilir.

Kosinüs benzerliği gibi metriklerle karşılaştırılır.

En yakın (benzer) belge(ler) seçilir.

Bu Kodda:
BERT kullanılarak belgeler ve sorgu için anlamsal (semantic) vektörler çıkarılıyor.

Sonra bu vektörler arasında cosine similarity hesaplanarak sorguya en yakın belge bulunuyor.

In [1]:
# gerekli kütüphaneleri içe aktar
from transformers import BertTokenizer, BertModel   # bERT tokenizer ve modeli
import numpy as np                                  # sayısal işlemler için NumPy
from sklearn.metrics.pairwise import cosine_similarity  # vektörler arası benzerlik için kosinüs benzerliği


In [2]:
# kullanılacak modelin adı (küçük boyutlu ve İngilizce, küçük harfli BERT modeli)
model_name = "bert-base-uncased"


In [3]:
# tokenizeri ve modeli önceden eğitilmiş ağırlıklarla yükle
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
# bilgi getirimi yapılacak metin belgeleri (documents)
documents = [
    "Machine learning is a field of artificial intelligence",
    "Natural language processing involves understanding human language",
    "Artificial intelligence encomppases machine learning and natural language processing (nlp)",
    "Deep learning is a subset of machine learning",
    "Data science combines statistics, adta analysis and machine learning",
    "I go to shop"  # alakasız bir cümle; sorgu ile düşük benzerlik beklenir
]


In [5]:
# kullanıcının sorduğu sorgu (query)
query = "What is deep learning?"

In [6]:
# bert ile metni sayısal vektöre çeviren fonksiyon
def get_embedding(text):
    # metni tokenize et, truncation ve padding uygula (model girişine uygun hale getir)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # modeli çalıştırarak çıktıları al
    outputs = model(**inputs)
    
    # modelin son gizli katmanını al (her token için vektör üretir)
    last_hidden_state = outputs.last_hidden_state
    
    # tüm token vektörlerinin ortalamasını alarak metnin genel temsili elde edilir
    embedding = last_hidden_state.mean(dim=1)  # [batch_size, hidden_dim]
    
    return embedding.detach().numpy()

In [7]:
# belgeler için embedding vektörlerini hesapla (hepsini üst üste diz)
doc_embeddings = np.vstack([get_embedding(doc) for doc in documents])

In [8]:
# sorgu query cümlesi için embedding hesapla
query_embedding = get_embedding(query)

In [9]:
# kosinüs benzerliği ile sorgu ve her belge arasındaki benzerlikleri hesapla
similarities = cosine_similarity(query_embedding, doc_embeddings)


In [10]:
# her belge için benzerlik skorlarını yazdır
for i, score in enumerate(similarities[0]):
    print(f"Document: {documents[i]} \n{score}\n")

Document: Machine learning is a field of artificial intelligence 
0.6348220109939575

Document: Natural language processing involves understanding human language 
0.6269391775131226

Document: Artificial intelligence encomppases machine learning and natural language processing (nlp) 
0.5046247243881226

Document: Deep learning is a subset of machine learning 
0.6263622641563416

Document: Data science combines statistics, adta analysis and machine learning 
0.613688588142395

Document: I go to shop 
0.5354948043823242



In [11]:
# en yüksek benzerlik skoruna sahip belgeyi bul
most_similar_index = similarities.argmax()

# en ilgili belgeyi ekrana yazdır
print(f"Most similar document: {documents[most_similar_index]}")

Most similar document: Machine learning is a field of artificial intelligence
